In [1]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
import ast
import time

In [2]:
def draw_image(X, y):
    # input: image and landmark coordinates
    # output: plot the labelled image
    X = X.reshape((256, 256, 3))
    y = 256 * y.reshape((194, 2))
    for i in range(y.shape[0]):
        for l in [-1, 0, 1]:
            for r in [-1, 0, 1]:
                X[y[i, 1] + l, y[i, 0] + r, 0] = 0
                X[y[i, 1] + l, y[i, 0] + r, 1] = 0
                X[y[i, 1] + l, y[i, 0] + r, 2] = 255
                
    plt.imshow(X[:,:,::-1])
    plt.show()

# get Y of an image
def load_y(file_prefix):
    # input: string; a prefix
    # output: np array of size 194 * 2; corresponding landmark coordinates
    path = 'D:\\Research\\Structure_1\\data\\'
    filename = path + file_prefix + '_coordinate'
    
    with open(filename, 'r') as f:
        facebox = f.readlines()
    facebox = ast.literal_eval(facebox[0])
    
    with open(path + file_prefix + '.txt', 'r') as f:
        index = f.readlines()
    
    coordinate = []

    for item in index[1:]:
        coordinate.append( list(map(float, item.strip('\n').split(','))))
        
    coordinate = np.round(np.array(coordinate)).astype(int)

    coordinate[:,0] -= facebox[0]
    coordinate[:,1] -= facebox[1]

    width = facebox[2] - facebox[0]
    height = facebox[3] - facebox[1]

    coordinate[:, 0] = coordinate[:, 0] / width * 256
    coordinate[:, 1] = coordinate[:, 1] / height * 256
    
    return coordinate.reshape((1, 194 * 2))

# get X of an image
def load_X(file_prefix):
    # input: string; a prefix
    # output: np array of size 194 * 194 * 3 by 1;
    path = 'D:\\Research\\Structure_1\\data\\'
    return cv2.imread(path + file_prefix + "_face.jpg").reshape((1, 256 * 256 * 3))

In [3]:
# load file prefix 
with open('filename.txt', 'r') as f:
    filename = f.readlines()

file_prefix_set = []
for item in filename:
    file_prefix_set.append(item.strip('\n'))

In [4]:
# Training Parameters
learning_rate = 0.0001
training_epoch = 200
batch_size = 20
display_step = 1
save_step = 10
training_size = 1800
validation_size = 200
test_size = 223

# Network Parameters
num_input = 256 # image size
num_channel = 3 # number of channel
num_output = 194 * 2 # coordinate size

# tf Graph input
tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, num_input * num_input * num_channel], name = 'X')
X_image = tf.reshape(X, [-1, num_input, num_input, num_channel])
y = tf.placeholder(tf.float32, [None, num_output], name = 'y')
keep_prob_1 = tf.placeholder(tf.float32, name = 'keep_prob_1')
keep_prob_2 = tf.placeholder(tf.float32, name = 'keep_prob_2')
keep_prob_3 = tf.placeholder(tf.float32, name = 'keep_prob_3')
keep_prob_4 = tf.placeholder(tf.float32, name = 'keep_prob_4')

def CNN(x):
    conv_1 = tf.layers.conv2d(inputs = x,
                              filters = 32,
                              kernel_size = [3, 3],
                              strides = (1, 1),
                              padding = 'valid',
                              activation = tf.nn.relu)
    
    pool_1 = tf.layers.max_pooling2d(inputs = conv_1,
                                     pool_size = [2, 2],
                                     strides = (2, 2),
                                     padding = 'valid')
    
    drop_out_1 = tf.nn.dropout(pool_1, keep_prob_1)
    
    conv_2 = tf.layers.conv2d(inputs = drop_out_1,
                              filters = 64,
                              kernel_size = [3, 3],
                              strides = (1, 1),
                              padding = 'valid',
                              activation = tf.nn.relu)
    
    conv_3 = tf.layers.conv2d(inputs = conv_2,
                              filters = 64,
                              kernel_size = [3, 3],
                              strides = (1, 1),
                              padding = 'valid',
                              activation = tf.nn.relu)
    
    pool_2 = tf.layers.max_pooling2d(inputs = conv_3,
                                     pool_size = [2, 2],
                                     strides = (2, 2),
                                     padding = 'valid')
    
    drop_out_2 = tf.nn.dropout(pool_2, keep_prob_2)
    
    conv_4 = tf.layers.conv2d(inputs = drop_out_2,
                              filters = 64,
                              kernel_size = [3, 3],
                              strides = (1, 1),
                              padding = 'valid',
                              activation = tf.nn.relu)
    
    conv_5 = tf.layers.conv2d(inputs = conv_4,
                              filters = 64,
                              kernel_size = [3, 3],
                              strides = (1, 1),
                              padding = 'valid',
                              activation = tf.nn.relu)
    
    pool_3 = tf.layers.max_pooling2d(inputs = conv_5,
                                     pool_size = [2, 2],
                                     strides = (2, 2),
                                     padding = 'valid')
    
    drop_out_3 = tf.nn.dropout(pool_3, keep_prob_3)

    flatten = tf.layers.flatten(inputs = drop_out_3)
    
    fc_1 = tf.layers.dense(inputs = flatten,
                           units = 1024,
                           activation = tf.nn.relu,
                           use_bias=True)
    
    drop_out_4 = tf.nn.dropout(fc_1, keep_prob_4)

    logits = tf.layers.dense(inputs = drop_out_4,
                             units = 194 * 2,
                             activation = None,
                             use_bias = True,
                             name = 'logits')
    
    return logits

In [5]:
logits = CNN(X_image)
label_tensor = tf.convert_to_tensor(y, dtype = tf.float32)
loss = tf.losses.mean_squared_error(labels = label_tensor, predictions = logits)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)
coordinate = tf.identity(logits, name = 'coordinate')

# Initialize the variables
init = tf.global_variables_initializer()

In [6]:
# total_index = np.random.permutation(len(file_prefix_set))
total_index = np.loadtxt('total_index.txt').astype('int')
training_index = total_index[:training_size]
validation_index = total_index[training_size : training_size + validation_size]
test_index = total_index[training_size + validation_size :]

In [ ]:
# record training loss and validation loss
trainRecord = []
validationRecord = []

# Training
saver = tf.train.Saver(max_to_keep = training_epoch)

start_time = time.time()
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epoch):
        
        # number of steps for each epoch
        total_batch = int(len(training_index) / batch_size)
        
        # shuffle index and create batches
        index = np.random.permutation(training_index)
        
        start = 0
        
        # go through every batch for training
        for i in range(total_batch):
            batch_X = np.zeros((batch_size, num_input * num_input * num_channel))
            batch_y = np.zeros((batch_size, num_output)) 
            
            for j in range(batch_size):
                batch_X[j, :] = load_X(file_prefix_set[index[i * batch_size + j]])
                batch_y[j, :] = load_y(file_prefix_set[index[i * batch_size + j]])

            sess.run(optimizer, feed_dict = {X: batch_X, y: batch_y, keep_prob_1 : 0.8, keep_prob_2 : 0.8, 
                                             keep_prob_3 : 0.8, keep_prob_4 : 0.8})
        
        if (epoch + 1) % display_step == 0:
            
            training_loss = 0
            for i in range(len(training_index)):
                train_X = load_X(file_prefix_set[training_index[i]])
                train_y = load_y(file_prefix_set[training_index[i]])
                training_loss += sess.run(loss, feed_dict = {X: train_X, y: train_y, keep_prob_1 : 1, keep_prob_2 : 1, 
                                                             keep_prob_3 : 1, keep_prob_4 : 1})
            trainRecord.append(training_loss ** 0.5 / len(training_index))

            validation_loss = 0
            for i in range(len(validation_index)):
                validation_X = load_X(file_prefix_set[validation_index[i]])
                validation_y = load_y(file_prefix_set[validation_index[i]])
                validation_loss += sess.run(loss, feed_dict = {X: validation_X, y: validation_y, keep_prob_1 : 1, keep_prob_2 : 1, 
                                                               keep_prob_3 : 1, keep_prob_4 : 1})
            validationRecord.append(validation_loss ** 0.5 / len(validation_index))
                        
            print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, trainRecord[-1], validationRecord[-1]))
        
            print('-----------------------------------------------------------------------------------------')
            
        if (epoch + 1) % save_step == 0:
            # Save model
            filename = 'D:\Research\Structure_1\model\CNN_Epoch_' + str(epoch + 1) + '.ckpt'
            saver.save(sess, filename)

print('Training Finished.')
end_time = time.time()
print('Training Time: {} seconds'.format(end_time - start_time))

Epoch: 0, Training Loss: 2.3515153939279014, Validation Loss: 7.068358505889005
-----------------------------------------------------------------------------------------
Epoch: 1, Training Loss: 2.3547213474112207, Validation Loss: 7.08250366598641
-----------------------------------------------------------------------------------------
Epoch: 2, Training Loss: 2.4173264631241653, Validation Loss: 7.272576214445736
-----------------------------------------------------------------------------------------
Epoch: 3, Training Loss: 2.2132385969505437, Validation Loss: 6.663903510064883
-----------------------------------------------------------------------------------------
Epoch: 4, Training Loss: 2.3004562164298927, Validation Loss: 6.9172513532601725
-----------------------------------------------------------------------------------------
Epoch: 5, Training Loss: 2.2573810001467933, Validation Loss: 6.7870359272105
------------------------------------------------------------------------

In [ ]:
import plotly.offline as offline
import plotly.graph_objs as go
offline.init_notebook_mode()

patternTrain = go.Scatter(x = list(range(len(trainRecord))), y = np.array(trainRecord), mode = 'markers + lines', name = 'Training')  
patternValidation = go.Scatter(x = list(range(len(validationRecord))), y = np.array(validationRecord), mode = 'markers + lines', name = 'Validation')  

# plot   
plot_data = [patternTrain, patternValidation]

layout = go.Layout(xaxis = dict(showgrid=True,
                              gridcolor='#bdbdbd',
                              gridwidth=0.1,
                              title = 'Epochs'),
                   yaxis = dict(showgrid=True,
                              gridcolor='#bdbdbd',
                              gridwidth=0.1,
                              title = 'Loss')
    )
fig = go.Figure(data = plot_data, layout = layout)
offline.iplot(fig)   

In [ ]:
np.argmin(np.array(validationRecord))